# Dataset

In [ ]:
# !pip install transformers
# !pip install datasets

In [7]:
path = "./data/train.gsv"
with open(path, 'r') as f:
    ls = f.readlines()

for l in ls:
    ts = l.strip().split('[|||]')
    print(ts);break
    if ts[1] == '':
        print(ts)
        input()

['166', '1', 'Episode No:  22A882047U', 'IDNUM: 22A882047U']


In [25]:
from datasets import load_dataset, Features, Value

dataset = load_dataset("csv", data_files="./data/train.gsv", delimiter='{{}}',
                       features = Features({
                              'fid': Value('string'), 'idx': Value('int64'),
                              'content': Value('string'), 'label': Value('string')}),
                              column_names=['fid', 'idx', 'content', 'label'], keep_default_na=False)

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 223.40it/s]
Generating train split: 0 examples [00:00, ? examples/s]/home/guest/r11922a16/miniconda3/envs/aicup23/lib/python3.10/site-packages/datasets/download/streaming_download_manager.py:765: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)
Generating train split: 143516 examples [00:00, 236754.59 examples/s]


In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['fid', 'idx', 'content', 'label'],
        num_rows: 143516
    })
})

In [26]:
dataset['train'][0]

{'fid': '166',
 'idx': 1,
 'content': 'Episode No:  22A882047U',
 'label': 'IDNUM: 22A882047U'}

In [27]:
dataset['train'][1]

{'fid': '166', 'idx': 25, 'content': '2208820.', 'label': 'PHI: NULL'}

In [28]:
dataset['train'][7]

{'fid': '166', 'idx': 114, 'content': 'Specimen: Tissue', 'label': 'PHI: NULL'}

For demonstration purpose, we only use the randomly sampled 20000 instances.

In [9]:
import torch
sub_datasets = torch.utils.data.random_split(dataset['train'], [20000, 65736])
print(len(sub_datasets[0]))
for i in range(4): print(sub_datasets[0][i])

20000
{'fid': '178', 'idx': 2810, 'content': 'Present (Extramural, A11)', 'label': 'PHI: NULL'}
{'fid': 'file68051', 'idx': 1716, 'content': 'E.', 'label': 'PHI: NULL'}
{'fid': '285', 'idx': 2350, 'content': 'Specimen labelled "Lesion right tongue base" consists of two biopsies 5 and 7mm.', 'label': 'PHI: NULL'}
{'fid': 'file4145', 'idx': 212, 'content': 'CLINICAL:', 'label': 'PHI: NULL'}


# Data loader

In [29]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

plm = "EleutherAI/pythia-70m" #"EleutherAI/pythia-70m-deduped"

bos = '<|endoftext|>'
eos = '<|END|>'
pad = '<|pad|>'
sep ='\n\n####\n\n'

special_tokens_dict = {'eos_token': eos, 'bos_token': bos, 'pad_token': pad, 'sep_token': sep}

tokenizer = AutoTokenizer.from_pretrained(plm, revision="step3000")
tokenizer.padding_side = 'left'
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
print(f"{tokenizer.pad_token}: {tokenizer.pad_token_id}")

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
tokenizer_config.json: 100%|██████████| 396/396 [00:00<00:00, 816kB/s]
tokenizer.json: 100%|██████████| 2.11M/2.11M [00:00<00:00, 2.15MB/s]
special_tokens_map.json: 100%|██████████| 99.0/99.0 [00:00<00:00, 163kB/s]

<|pad|>: 50278


In [11]:
!pip install islab-opendeid

In [12]:
from torch.utils.data import DataLoader
from islab.aicup import collate_batch_with_prompt_template

train_data = list(sub_datasets[0])
train_dataloader = DataLoader(train_data, batch_size=3, shuffle=False, collate_fn=lambda batch: collate_batch_with_prompt_template(batch, tokenizer))
titer = iter(train_dataloader)
tks, labels, masks= next(titer)
print(tks.shape)
next(iter(titer))

torch.Size([3, 27])


(tensor([[50278, 50278, 50278,     0,  8951,  1042, 25129,    27, 50279,  6663,
             42,    27,  5812,   209, 50277],
         [    0,  1707,   310,   247, 43278,  4182,  3691,    15, 50279,  6663,
             42,    27,  5812,   209, 50277],
         [    0, 36859,  1610,   361, 50279, 19047, 25129, 34181,  8242,    27,
          36859,  1610,   361,   209, 50277]]),
 tensor([[ -100,  -100,  -100,     0,  8951,  1042, 25129,    27, 50279,  6663,
             42,    27,  5812,   209, 50277],
         [    0,  1707,   310,   247, 43278,  4182,  3691,    15, 50279,  6663,
             42,    27,  5812,   209, 50277],
         [    0, 36859,  1610,   361, 50279, 19047, 25129, 34181,  8242,    27,
          36859,  1610,   361,   209, 50277]]),
 tensor([[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]))

In [13]:
results = tokenizer(["Lab No: 14H02780", "“STOCKDALE” 653 MONAGHAN RD"], padding=True)
print(results['input_ids'])
print()
print(results['input_ids'][0])
print(tokenizer.decode(results['input_ids'][0]))
print(results['input_ids'][1])
print(tokenizer.decode(results['input_ids'][1]))

[[50278, 50278, 50278, 50278, 50278, 50278, 21663, 1621, 27, 1638, 41, 16604, 1438], [1628, 1267, 9466, 37, 23502, 668, 721, 3357, 33995, 2696, 41, 1539, 28613]]

[50278, 50278, 50278, 50278, 50278, 50278, 21663, 1621, 27, 1638, 41, 16604, 1438]
<|pad|><|pad|><|pad|><|pad|><|pad|><|pad|>Lab No: 14H02780
[1628, 1267, 9466, 37, 23502, 668, 721, 3357, 33995, 2696, 41, 1539, 28613]
“STOCKDALE” 653 MONAGHAN RD


In [14]:
results = tokenizer(
    [f"{bos} 9364819.RAN\\nMINTANIA, JEFFRY {sep} ID: 9364819.RAN\\nNAME: MINTANIA, JEFFRY {eos}",
     f"{bos} This is a sentence {sep} PHI: NULL {eos}"],
    padding=True
)
print(results['attention_mask'][0])
print(results['attention_mask'][1])
print(tokenizer.decode(results['input_ids'][0]))
print(tokenizer.decode(results['input_ids'][1]))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
<|endoftext|> 9364819.RAN\nMINTANIA, JEFFRY 

####

 ID: 9364819.RAN\nNAME: MINTANIA, JEFFRY <|END|>
<|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|endoftext|> This is a sentence 

####

 PHI: NULL <|END|>


In [15]:
from islab.aicup import OpenDeidBatchSampler

BATCH_SIZE = 8
bucket_train_dataloader = DataLoader(train_data,
                                     batch_sampler=OpenDeidBatchSampler(train_data, BATCH_SIZE),
                                     collate_fn=lambda batch: collate_batch_with_prompt_template(batch, tokenizer),
                                     pin_memory=True)

# for idx, batch in enumerate(bucket_train_dataloader):
#     print(batch)
#     print(batch[0].shape)
#     print(batch[1].shape)
#     break

# Model

In [16]:
from transformers import AutoConfig
# the model config to which we add the special tokens
config = AutoConfig.from_pretrained(plm,
                                    bos_token_id=tokenizer.bos_token_id,
                                    eos_token_id=tokenizer.eos_token_id,
                                    pad_token_id=tokenizer.pad_token_id,
                                    sep_token_id=tokenizer.sep_token_id,
                                    output_hidden_states=False)

model = AutoModelForCausalLM.from_pretrained(plm, revision="step3000", config=config)
model

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
          (a

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [18]:
from transformers import get_linear_schedule_with_warmup
from torch.optim import AdamW

EPOCHS = 3 # CHANGE TO THE NUMBER OF EPOCHS YOU WANT
optimizer = AdamW(model.parameters(),lr=3e-5) # YOU CAN ADJUST LEARNING RATE

model.resize_token_embeddings(len(tokenizer))
model.to(device)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50280, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
          (a

In [19]:
from tqdm import tqdm,trange

global_step = 0
total_loss = 0

model.train()
for _ in trange(EPOCHS, desc="Epoch"):
    model.train()
    total_loss = 0

    # Training loop
    predictions , true_labels = [], []

    for step, (seqs, labels, masks) in enumerate(bucket_train_dataloader):
        seqs = seqs.to(device)
        labels = labels.to(device)
        masks = masks.to(device)
        model.zero_grad()
        outputs = model(seqs, labels=labels, attention_mask=masks)
        logits = outputs.logits
        loss = outputs.loss
        loss = loss.mean()

        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    avg_train_loss = total_loss / len(bucket_train_dataloader)
    print("Average train loss: {}".format(avg_train_loss))

Epoch:  33%|███▎      | 1/3 [01:49<03:39, 109.76s/it]

Average train loss: 1.875850281918049


Epoch:  67%|██████▋   | 2/3 [03:35<01:47, 107.22s/it]

Average train loss: 1.4357917549610137


Epoch: 100%|██████████| 3/3 [05:21<00:00, 107.00s/it]

Average train loss: 1.2528684722542762


In [20]:
from datasets import load_dataset, Features, Value
valid_data = load_dataset("csv", data_files="/content/drive/MyDrive/aicup/PublicDataset_phase3/opendid_valid.tsv", delimiter='\t',
                          features = Features({
                              'fid': Value('string'), 'idx': Value('int64'),
                              'content': Value('string'), 'label': Value('string')}),
                              column_names=['fid', 'idx', 'content', 'label'])
valid_list= list(valid_data['train'])
valid_list

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

[{'fid': '1001',
  'idx': 0,
  'content': 'Episode No:  88Y206206L',
  'label': None},
 {'fid': '1001', 'idx': 24, 'content': '8892062.BPL', 'label': None},
 {'fid': '1001',
  'idx': 37,
  'content': 'Vatterott, Jerrie CLARENCE',
  'label': None},
 {'fid': '1001',
  'idx': 65,
  'content': 'Lab No:  88Y20620,88Y20620',
  'label': None},
 {'fid': '1001', 'idx': 92, 'content': 'Exeter', 'label': None},
 {'fid': '1001',
  'idx': 99,
  'content': 'DECEPTION BAY  Northern Territory  6845',
  'label': None},
 {'fid': '1001',
  'idx': 139,
  'content': 'Specimen: Fluid,Tissue',
  'label': None},
 {'fid': '1001', 'idx': 162, 'content': 'D.O.B:  15/11/2004', 'label': None},
 {'fid': '1001', 'idx': 181, 'content': 'Sex:  F', 'label': None},
 {'fid': '1001',
  'idx': 189,
  'content': 'Collected: 20/5/2064 at :',
  'label': None},
 {'fid': '1001',
  'idx': 215,
  'content': 'Location:  PARKES 8 - GUNNEDAH DISTRICT HOSPITAL',
  'label': None},
 {'fid': '1001',
  'idx': 264,
  'content': 'DR Edison

In [21]:
from tqdm.notebook import tqdm
from islab.aicup import aicup_predict
import io
BATCH_SIZE = 32

with open("./answer.txt",'w',encoding='utf8') as f:
#with io.open("answer.txt",'w',encoding='utf8') as f:
    for i in tqdm(range(0, len(valid_list), BATCH_SIZE)):
        with torch.no_grad():
            seeds = valid_list[i:i+BATCH_SIZE]
            outputs = aicup_predict(model, tokenizer, input=seeds)
            for o in outputs:
                f.write(o)
                f.write('\n')

  0%|          | 0/1387 [00:00<?, ?it/s]